In [52]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

This file creates the trade file behind the [Phase One Tracker](https://www.tradewartracker.com/). It proceeds in several steps.

1. Grabs the trade data

2. Constructs the relavent Phase One product catagories and the associated goals/targets.

3. Maps the data to the county level.

In [53]:
trade_type = "exports"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

---
#### Grabe the trade data using the Census's API

In [54]:
def census_trade(url, trade_type, country, product_level):
    
    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m")
    # This is so I can call this correctly...
    
    if trade_type == "imports":
        
        trade_label = country + "_" + trade_type
        
        df[trade_label] = df["CON_VAL_MO"].astype(float)
        
        df[product_level] = df["I_COMMODITY"].astype(str)
        
        df.drop(["CON_VAL_MO", "I_COMMODITY", "COMM_LVL"], axis = 1, inplace = True)
        
    if trade_type == "exports":
    
        trade_label = country + "_" + trade_type
        
        df[trade_label] = df["ALL_VAL_MO"].astype(float)

        df[product_level] = df["E_COMMODITY"].astype(str)
        
        df["china_exports_quant"] = df["QTY_1_MO"].astype(float)
    
        df["quant_type"] = df["UNIT_QY1"].astype(str)
        
        df.drop(["ALL_VAL_MO", "E_COMMODITY", "COMM_LVL", "QTY_1_MO", "UNIT_QY1"], axis = 1, inplace = True)
    
    return df

In [55]:
end_use = "hs?get=E_COMMODITY,ALL_VAL_MO,UNIT_QY1,QTY_1_MO"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
url = url + my_key + "&time==from+2013-01+to+2020-12" + "&COMM_LVL=HS6"

url = url + "&CTY_CODE=5700"

df = census_trade(url, trade_type, "china", "hs6")

df["hs4"] = df["hs6"].str[0:4]

df.time.max()

<Response [200]>


Timestamp('2020-12-01 00:00:00')

In [56]:
df.tail()

,time,CTY_CODE,china_exports,hs6,china_exports_quant,quant_type,hs4
363089,2020-12-01,5700,0.0,722011,0.0,-,7220
363090,2020-12-01,5700,0.0,722710,0.0,-,7227
363091,2020-12-01,5700,21993.0,722920,0.0,-,7229
363092,2020-12-01,5700,0.0,730230,0.0,-,7302
363093,2020-12-01,5700,30872.0,730690,0.0,-,7306


---

Now this is the **new** stuff from the Census. The plan is to append it to the regular data set...

In [57]:
#census_yrl = "https://www.census.gov/foreign-trade/Press-Release/2020pr/06/Prelim_US_C_ETA/June%202020.xlsx"
census_yrl = "https://www.census.gov/foreign-trade/Press-Release/2021pr/01/Prelim_US_C_ETA/January%202021.xlsx"

dfC = pd.read_excel(census_yrl, skiprows = [0,1])

dfC.columns = ["hs4_o", "C_description", "china_exports"]

dfC["time"] = "2021-01-01"

dfC.time = pd.to_datetime(dfC.time, format="%Y-%m-%d")

dfC.dropna(inplace = True)

dfC.tail()

,hs4_o,C_description,china_exports,time
549,9018,"Instruments And Appliances Used In Medical, Su...",255395084.0,2021-01-01
550,9019,"Mechano-Therapy, Massage, Psychological Aptitu...",6382080.0,2021-01-01
551,9020,Breathing Appliances Nesoi And Gas Masks Havin...,2855871.0,2021-01-01
552,9021,Orthopedic Appliances; Splints Etc.; Artificia...,70906120.0,2021-01-01
553,9022,"X-Ray Etc. Apparatus, Including Radiography Or...",53930228.0,2021-01-01


In [58]:
dfproducts = pd.read_csv(".\\data"+ "\\annex-6-1.csv", dtype = {"hs4": str, "hs4_o": str})

In [59]:
dfC = dfC[["hs4_o","china_exports","time"]].merge(dfproducts, left_on = "hs4_o", right_on = "hs4_o", how = "left")

In [60]:
dfC.head()

,hs4_o,china_exports,time,description,low_catagory,high_catagory,hs4
0,0101,0.0,2021-01-01,"Hors,es ass,es mules and hinnies, live",Other agricultural commodities,2. Agriculture,0101
1,0102,0.0,2021-01-01,"Bovine anima,ls live",Other agricultural commodities,2. Agriculture,0102
2,0103,1970000.0,2021-01-01,"Swine, live",Other agricultural commodities,2. Agriculture,0103
3,0104,0.0,2021-01-01,"Sheep and goa,ts live",Other agricultural commodities,2. Agriculture,0104
4,0105,0.0,2021-01-01,"Poultry, live; chickens, ducks, geese, turkeys...",Other agricultural commodities,2. Agriculture,0105


---
#### Bring in the Phase One Product list

In [61]:
not_energy = dfproducts.high_catagory != "3. Energy"

energy_list = list(dfproducts.loc[~not_energy,"hs4"])

print(energy_list)

['2701', '2709', '2710', '2711', '2711', '2711', '2711', '2713', '2713', '2905']


Next step, we will remove the energy products, these will be added on below from the ``phase-one-energy`` notebook

In [62]:
df = df[df["hs4"].isin(energy_list) == False]

Now merge the non energy products with the product lists

In [63]:
df_phaseone = df.merge(dfproducts, left_on = "hs4", right_on = "hs4", how = "left", indicator = True)

Grabe the column names so the energy data will be the same order otherwise pandas complains

In [64]:
cnames = list(df_phaseone.columns)
print(cnames)

['time', 'CTY_CODE', 'china_exports', 'hs6', 'china_exports_quant', 'quant_type', 'hs4', 'hs4_o', 'description', 'low_catagory', 'high_catagory', '_merge']


Then read in the energy data set

In [65]:
file = ".\\data"+ "\\phaseone-energy.parquet"

df_energy = pq.read_table(file).to_pandas()

df_energy[df_energy.low_catagory == "Liquefied natural gas"]

,time,CTY_CODE,hs4_o,hs6,china_exports,china_exports_quant,quant_type,description,low_catagory,high_catagory,hs4,_merge
575,2013-04-01,5700,271111,271111,119031.0,996.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
576,2013-05-01,5700,271111,271111,0.0,0.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
577,2013-06-01,5700,271111,271111,0.0,0.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
578,2013-07-01,5700,271111,271111,0.0,0.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
579,2013-08-01,5700,271111,271111,0.0,0.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
...,...,...,...,...,...,...,...,...,...,...,...,...
642,2020-08-01,5700,271111,271111,52565450.0,318840.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
643,2020-09-01,5700,271111,271111,47949278.0,340983.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
644,2020-10-01,5700,271111,271111,198693670.0,1652567.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both
645,2020-11-01,5700,271111,271111,284244539.0,2097456.0,M3,Liquefied natural gas,Liquefied natural gas,3. Energy,2711,both


And then append it to the main dataframe

In [66]:
df_phaseone = df_phaseone.append(df_energy)

In [67]:
df_phaseone = df_phaseone.append(dfC)

In [68]:
df_phaseone.tail()

,time,CTY_CODE,china_exports,hs6,china_exports_quant,quant_type,hs4,hs4_o,description,low_catagory,high_catagory,_merge
549,2021-01-01,NaN,255395084.0,NaN,NaN,NaN,9018,9018,"Instruments and appliances used in medic,al su...",Optical and medical instruments,1. Manufactured Goods,NaN
550,2021-01-01,NaN,6382080.0,NaN,NaN,NaN,9019,9019,Mechano-therapy appliances; massage apparatus;...,Optical and medical instruments,1. Manufactured Goods,NaN
551,2021-01-01,NaN,2855871.0,NaN,NaN,NaN,9020,9020,"Other breathing appliances and gas masks, excl...",Optical and medical instruments,1. Manufactured Goods,NaN
552,2021-01-01,NaN,70906120.0,NaN,NaN,NaN,9021,9021,"Orthopedic appliances, including crutch,es su...",Optical and medical instruments,1. Manufactured Goods,NaN
553,2021-01-01,NaN,53930228.0,NaN,NaN,NaN,9022,9022,Apparatus based on the use of X-rays or of alp...,Optical and medical instruments,1. Manufactured Goods,NaN


In [69]:
# then we can see the stuff with the orginal code
df_phaseone[df_phaseone.hs4_o == "271112"]

,time,CTY_CODE,china_exports,hs6,china_exports_quant,quant_type,hs4,hs4_o,description,low_catagory,high_catagory,_merge
647,2013-10-01,5700,49461021.0,271112,1105106.0,BBL,2711,271112,Liquefied propane,Refined products,3. Energy,both
648,2013-11-01,5700,20872845.0,271112,388600.0,BBL,2711,271112,Liquefied propane,Refined products,3. Energy,both
649,2013-12-01,5700,70036260.0,271112,1154000.0,BBL,2711,271112,Liquefied propane,Refined products,3. Energy,both
650,2014-02-01,5700,39743760.0,271112,577000.0,BBL,2711,271112,Liquefied propane,Refined products,3. Energy,both
651,2014-03-01,5700,29323140.0,271112,577000.0,BBL,2711,271112,Liquefied propane,Refined products,3. Energy,both
...,...,...,...,...,...,...,...,...,...,...,...,...
739,2020-09-01,5700,7178500.0,271112,46583.0,M3,2711,271112,Liquefied propane,Refined products,3. Energy,both
740,2020-10-01,5700,0.0,271112,0.0,M3,2711,271112,Liquefied propane,Refined products,3. Energy,both
741,2020-11-01,5700,0.0,271112,0.0,M3,2711,271112,Liquefied propane,Refined products,3. Energy,both
742,2020-12-01,5700,0.0,271112,0.0,M3,2711,271112,Liquefied propane,Refined products,3. Energy,both


In [70]:
df_phaseone.high_catagory = df_phaseone.high_catagory.fillna("not in aggreement")

Then the outfiles ``phaseone-tradedata.parquet`` is the main file used in ``phase-one-plots.ipynb`` notebook

In [71]:
df_phaseone.china_exports.sum()

962961861096.0

In [72]:
df_phaseone.high_catagory.replace({"1. Manufactured Goods": "Manufactures", "2. Agriculture": "Agriculture", "3. Energy": "Energy"}, inplace = True)

out_file = ".\\data"+ "\\phaseone-tradedata-C-january.parquet"

pq.write_table(pa.Table.from_pandas(df_phaseone), out_file)

#### This then constructs the benchmark and goal measures

In [73]:
df_phaseone.set_index("time", inplace = True)

In [74]:
grp = df_phaseone.loc["2017"].groupby("high_catagory")

benchmarks = grp.agg({"china_exports": "sum"})

benchmarks.columns = ["2017 Values"]

In [75]:
grp = df_phaseone.loc["2020"].groupby("high_catagory")

current = grp.agg({"china_exports": "sum"})

current.columns = ["2020 Values"]

In [76]:
benchmarks = benchmarks.merge(current, left_index = True, right_index = True)

In [77]:
grp = df_phaseone.loc["2021"].groupby("high_catagory")

current = grp.agg({"china_exports": "sum"})

current.columns = ["2021 Values"]

In [78]:
benchmarks = benchmarks.merge(current, left_index = True, right_index = True)

In [79]:
benchmarks

,2017 Values,2020 Values,2021 Values
high_catagory,,,
Agriculture,2.096979e+10,2.733534e+10,3.864486e+09
Energy,7.649222e+09,9.738383e+09,1.178109e+09
Manufactures,5.015148e+10,5.217981e+10,4.756725e+09
not in aggreement,5.026467e+10,3.487364e+10,1.782098e+07


In [80]:
# These are the goals from the AGREEMENT

benchmarks["2020 Goals"] = np.nan

benchmarks.at["Agriculture","2020 Goals"] = 12500000000

benchmarks.at["Energy","2020 Goals"] = 18500000000

benchmarks.at["Manufactures","2020 Goals"] = 32900000000 

#benchmarks.at["not in aggreement","2020 Goals"] = np.nan

Then the ``phaseone-goals.parquet`` file is the data file used to create the bar graph.

In [81]:
benchmarks

,2017 Values,2020 Values,2021 Values,2020 Goals
high_catagory,,,,
Agriculture,2.096979e+10,2.733534e+10,3.864486e+09,1.250000e+10
Energy,7.649222e+09,9.738383e+09,1.178109e+09,1.850000e+10
Manufactures,5.015148e+10,5.217981e+10,4.756725e+09,3.290000e+10
not in aggreement,5.026467e+10,3.487364e+10,1.782098e+07,NaN


In [82]:
benchmarks["2021 Goals"] = np.nan

benchmarks.at["Agriculture","2021 Goals"] = 19500000000

benchmarks.at["Energy","2021 Goals"] = 33900000000 

benchmarks.at["Manufactures","2021 Goals"] = 44800000000

In [83]:
out_file = ".\\data"+ "\\phaseone-goals-C-january.parquet"

pq.write_table(pa.Table.from_pandas(benchmarks), out_file)

In [84]:
benchmarks

,2017 Values,2020 Values,2021 Values,2020 Goals,2021 Goals
high_catagory,,,,,
Agriculture,2.096979e+10,2.733534e+10,3.864486e+09,1.250000e+10,1.950000e+10
Energy,7.649222e+09,9.738383e+09,1.178109e+09,1.850000e+10,3.390000e+10
Manufactures,5.015148e+10,5.217981e+10,4.756725e+09,3.290000e+10,4.480000e+10
not in aggreement,5.026467e+10,3.487364e+10,1.782098e+07,NaN,NaN


In [85]:
df_phaseone.high_catagory.replace({"1. Manufactured Goods": "Manufactures", "2. Agriculture": "Agriculture", "3. Energy": "Energy"}, inplace = True)

In [86]:
df_phaseone

,CTY_CODE,china_exports,hs6,china_exports_quant,quant_type,hs4,hs4_o,description,low_catagory,high_catagory,_merge
time,,,,,,,,,,,
2013-01-01,5700,287000.0,481031,0.0,-,4810,NaN,NaN,NaN,not in aggreement,left_only
2013-01-01,5700,311650.0,481160,0.0,-,4811,NaN,NaN,NaN,not in aggreement,left_only
2013-01-01,5700,63701.0,481620,0.0,-,4816,NaN,NaN,NaN,not in aggreement,left_only
2013-01-01,5700,40265.0,490290,0.0,-,4902,NaN,NaN,NaN,not in aggreement,left_only
2013-01-01,5700,324939.0,480431,0.0,-,4804,NaN,NaN,NaN,not in aggreement,left_only
...,...,...,...,...,...,...,...,...,...,...,...
2021-01-01,NaN,255395084.0,NaN,NaN,NaN,9018,9018,"Instruments and appliances used in medic,al su...",Optical and medical instruments,Manufactures,NaN
2021-01-01,NaN,6382080.0,NaN,NaN,NaN,9019,9019,Mechano-therapy appliances; massage apparatus;...,Optical and medical instruments,Manufactures,NaN
2021-01-01,NaN,2855871.0,NaN,NaN,NaN,9020,9020,"Other breathing appliances and gas masks, excl...",Optical and medical instruments,Manufactures,NaN
